<a href="https://colab.research.google.com/github/jkf87/autogen-handson/blob/main/%5Blangchain%5Dmap%2C_reduce_%EC%9C%A0%ED%8A%9C%EB%B8%8C_%EC%98%81%EC%83%81_%EC%9A%94%EC%95%BD_%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Video Summarization
- 유튜브 비디오 내용 요약 인공지능 만들기


1. 유튜브 영상 다운로드 (음성만 다운로드, mp3) - PyTube
2. Speech to Text (Transcribe) - OpenAI Whisper (Local)
3. **Map-reduce** summariation - LangChain, OpenAI ChatGPT API


### 출처 유튜브 [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)


In [ ]:
!sudo apt update && sudo apt install ffmpeg
!pip install -q openai-whisper pytube
!pip install -q openai tiktoken langchain

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,269 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,347 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,957 kB in 2s (1,297 kB/s)
Reading package li

## Download YouTube video

[ChatGPT 파인튜닝 심리상담 챗봇 만들기 (비용, 시간, 데이터셋, 방법 소개)](https://youtu.be/sTC_srIwhIw?si=znP9FavxST5p0RFZ) - 빵형의 개발도상국 유튜브 채널

In [ ]:
from pytube import YouTube

yt = YouTube('https://youtu.be/sTC_srIwhIw?si=_BHcN3eH3lFhtdtZ')

yt.streams.filter(only_audio=True).first().download(
    output_path='.', filename='input.mp3')

'/content/./input.mp3'

## Transcribe

In [ ]:
import whisper

model = whisper.load_model("base")

30초 이상 오디오 데이터 처리 방법

https://github.com/openai/whisper/discussions/136

In [ ]:
result = model.transcribe("input.mp3")

result["text"][:300]

" This is the video where we are going to discuss GPT-4 vision prompt engineering techniques. At this point, there are not a lot of tutorials on GPT-4 vision prompt engineering techniques, but Microsoft recently put together very detailed paper on GPT-4 vision that's called the dawn of the LMM's. The"

## Map-reduce summarization

https://python.langchain.com/docs/use_cases/summarization#option-2-map-reduce

한글 설명: https://teddylee777.github.io/langchain/langchain-tutorial-07/

![](https://i.imgur.com/2jJ0unq.jpg)

### Text splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

# 이 객체는 텍스트를 나누어 주어진 청크 크기에 따라 여러 부분으로 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 길이를 설정합니다.
    chunk_overlap=50,  # 청크 간에 겹치는 문자 수를 설정합니다.
    length_function=len,  # 청크 길이를 측정하는 함수를 설정합니다.
)

# 텍스트를 분할하고, 각 부분을 Document 객체로 변환합니다.
docs = [Document(page_content=x) for x in text_splitter.split_text(result["text"])]

# 분할된 문서들을 더욱 작은 부분들로 나눕니다.
split_docs = text_splitter.split_documents(docs)

# 분할된 문서의 개수를 출력합니다.
len(split_docs)


17

In [ ]:
split_docs

[Document(page_content="This is the video where we are going to discuss GPT-4 vision prompt engineering techniques. At this point, there are not a lot of tutorials on GPT-4 vision prompt engineering techniques, but Microsoft recently put together very detailed paper on GPT-4 vision that's called the dawn of the LMM's. The large multi-model models. From that paper, there is a particular segment that discusses about prompt engineering techniques, which we are going to go over in this video. There are going to be four different kinds of prompt engineering techniques. The first one is called following text instructions. The second one is called visual pointing or visual referral prompting and then the third one is called few short in prompt learning or in context learning and then the final one fourth one is the visual plus text, the combination of both and we are going to discuss all these four with the examples that Microsoft has given us in this paper. Let's get started. The first one y

### Prompts

In [ ]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

openai_api_key = "sk-"

llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

# Map prompt
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""

map_prompt = PromptTemplate.from_template(map_template)

# Reduce prompt
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes.
The final answer is a single paragraph of about 300 words and must be in Korean.
Helpful Answer:"""

reduce_prompt = PromptTemplate.from_template(reduce_template)

### Map-Reduce chains

In [ ]:
# 1. 체인 축소
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 문서 리스트를 받아서 하나의 문자열로 결합하고, 이를 LLMChain에 전달
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# 매핑된 문서를 결합하고 반복적으로 축소
reduce_documents_chain = ReduceDocumentsChain(
    # 이것은 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 `StuffDocumentsChain`의 컨텍스트를 초과하면
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 최대 토큰 수.
    token_max=4000,
)

# 2. 맵 체인
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# 체인을 매핑하여 문서를 결합하고, 결과를 결합
map_reduce_chain = MapReduceDocumentsChain(
    # 맵 체인
    llm_chain=map_chain,
    # 리듀스 체인
    reduce_documents_chain=reduce_documents_chain,
    # llm_chain에 문서를 넣을 변수 이름
    document_variable_name="docs",
    # 출력에서 맵 단계의 결과를 반환
    return_intermediate_steps=False,
)


![image.png](https://i.imgur.com/za0cKc2.png)

### Run

In [ ]:
sum_result = map_reduce_chain.run(split_docs)

print(sum_result)

주어진 문서들에서 주요 주제는 GPT-4 비전 프롬프트 엔지니어링 기술, 텍스트 지침 따르기, 시각적 지시 또는 시각적 참조 프롬프팅, 퓨-샷 또는 문맥 속 학습, 그리고 시각과 텍스트의 결합입니다. 또한, 식사와 음식 관련 경험, 야외와 해안 지역 설정, 휴식과 평온, 시각적 요소 등의 주제도 확인할 수 있습니다. 이러한 주제들은 주어진 정보를 기반으로 추론되었으며, 문서에 존재하는 모든 주제를 포괄하지 않을 수 있습니다.


## UI using Gradio

외국어 -> 한국어 가능

https://www.youtube.com/watch?v=BAfOGBojiEU

![](https://i.imgur.com/sm2W5jE.png)

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr
import re

# 비디오 ID를 추출하는 함수 정의
def extract_video_id(url):
    # YouTube URL에서 비디오 ID를 추출하기 위한 정규 표현식
    youtube_regex = (r'(https?://)?(www\.)?'
        '(youtube|youtu|youtube-nocookie)\.(com|be)/'
        '(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})')
    youtube_pattern = re.compile(youtube_regex)
    match = youtube_pattern.match(url)  # 정규 표현식과 URL이 매치되는지 확인
    if not match:  # 매치되지 않으면 None 반환
        return None
    return match.group(6)  # 매치된 비디오 ID 반환

# 요약 함수 정의
def summarize(url):
    yt = YouTube(url)  # YouTube 객체 생성

    # 오디오 스트림만 필터링하고 첫 번째 오디오 스트림을 다운로드
    yt.streams.filter(only_audio=True).first().download(
        output_path='.', filename='input.mp3')

    result = model.transcribe("input.mp3")  # 오디오 파일을 텍스트로 변환

    # 텍스트를 분할하고 각 부분을 Document 객체로 변환
    docs = [Document(page_content=x) for x in text_splitter.split_text(result["text"])]
    split_docs = text_splitter.split_documents(docs)  # 분할된 문서들을 더 작은 부분으로 나눔

    sum_result = map_reduce_chain.run(split_docs)  # 분할된 문서들을 처리하고 요약

    video_id = extract_video_id(url)  # 비디오 ID 추출
    # YouTube 비디오를 임베드하는 HTML 코드 생성
    embed = f"""<iframe width='560' height='315' src='https://www.youtube.com/embed/{video_id}' frameborder='0' allowfullscreen></iframe>"""

    return sum_result, embed  # 요약 결과와 임베드 코드 반환

# Gradio 인터페이스 생성
demo = gr.Interface(
    fn=summarize,  # 요약 함수 설정
    inputs=gr.Textbox(label="URL"),  # URL 입력 텍스트 박스
    outputs=[gr.TextArea(label="Summary"), gr.HTML()],  # 요약과 HTML 출력 영역
)

# 대기열을 추가하고 인터페이스를 실행
demo.queue().launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://13befeb15c750212da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/components/button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(


Keyboard interruption in main thread... closing server.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/whisper/audio.py", line 59, in load_audio
    out = run(cmd, capture_output=True, check=True).stdout
  File "/usr/lib/python3.10/subprocess.py", line 526, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ffmpeg', '-nostdin', '-threads', '0', '-i', 'input.mp3', '-f', 's16le', '-ac', '1', '-acodec', 'pcm_s16le', '-ar', '16000', '-']' returned non-zero exit status 255.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 406, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1554, in process_api

Killing tunnel 127.0.0.1:7860 <> https://13befeb15c750212da.gradio.live


이렇게 만든 코드를 허깅페이스에 업로드하기
https://www.phind.com/agent?cache=clnq2we7w0004l908wcpuuhhy
